In [2]:
1 + 1

2

In [5]:
# Chain
[1, 4, 1, 3, 2] |> 
    sort |> 
    unique

4-element Array{Int64,1}:
 1
 2
 3
 4

In [5]:
using Agents, AgentsPlots

┌ Info: Precompiling AgentsPlots [7820620d-95fb-4739-93b3-b0a14dd83f9a]
└ @ Base loading.jl:1260


InterruptException: InterruptException:

In [2]:
mutable struct SchellingAgent <: AbstractAgent
    id::Int # The identifier number of the agent
    pos::Tuple{Int,Int} # The x, y location of the agent on a 2D grid
    mood::Bool # whether the agent is happy in its node. (true = happy)
    group::Int # The group of the agent,  determines mood as it interacts with neighbors
end

In [3]:
space = GridSpace((10, 10), moore = true)

GridSpace with 100 nodes and 342 edges

In [4]:
properties = Dict(:min_to_be_happy => 3)
schelling = ABM(SchellingAgent, space; properties = properties)

AgentBasedModel with 0 agents of type SchellingAgent
 space: GridSpace with 100 nodes and 342 edges
 scheduler: fastest
 properties: Dict(:min_to_be_happy => 3)

In [7]:
function initialize(; numagents = 320, griddims = (20, 20), min_to_be_happy = 3)
    space = GridSpace(griddims, moore = true)
    properties = Dict(:min_to_be_happy => min_to_be_happy)
    model =
        ABM(SchellingAgent, space; properties = properties, scheduler = random_activation)
    # populate the model with agents, adding equal amount of the two types of agents
    # at random positions in the model
    for n in 1:numagents
        agent = SchellingAgent(n, (1, 1), false, n < numagents / 2 ? 1 : 2)
        add_agent_single!(agent, model)
    end
    return model
end

initialize (generic function with 1 method)

In [8]:
function agent_step!(agent, model)
    agent.mood == true && return # do nothing if already happy
    minhappy = model.min_to_be_happy
    neighbor_cells = node_neighbors(agent, model)
    count_neighbors_same_group = 0
    # For each neighbor, get group and compare to current agent's group
    # and increment count_neighbors_same_group as appropriately.
    for neighbor_cell in neighbor_cells
        node_contents = get_node_contents(neighbor_cell, model)
        # Skip iteration if the node is empty.
        length(node_contents) == 0 && continue
        # Otherwise, get the first agent in the node...
        agent_id = node_contents[1]
        # ...and increment count_neighbors_same_group if the neighbor's group is
        # the same.
        neighbor_agent_group = model[agent_id].group
        if neighbor_agent_group == agent.group
            count_neighbors_same_group += 1
        end
    end
    # After counting the neighbors, decide whether or not to move the agent.
    # If count_neighbors_same_group is at least the min_to_be_happy, set the
    # mood to true. Otherwise, move the agent to a random node.
    if count_neighbors_same_group ≥ minhappy
        agent.mood = true
    else
        move_agent_single!(agent, model)
    end
    return
end

agent_step! (generic function with 1 method)

In [9]:
model = initialize()

AgentBasedModel with 320 agents of type SchellingAgent
 space: GridSpace with 400 nodes and 1482 edges
 scheduler: random_activation
 properties: Dict(:min_to_be_happy => 3)

In [10]:
step!(model, agent_step!)

In [11]:
step!(model, agent_step!, 3)

In [12]:
properties = [:pos, :mood, :group]

model = initialize()
data, _ = run!(model, agent_step!, 5; adata = properties)
data[1:10, :] # print only a few rows

,step,id,pos,mood,group
,Int64,Int64,Tuple…,Bool,Int64
1,0,1,"(10, 9)",0,1
2,0,2,"(19, 13)",0,1
3,0,3,"(7, 6)",0,1
4,0,4,"(14, 5)",0,1
5,0,5,"(7, 1)",0,1
6,0,6,"(19, 17)",0,1
7,0,7,"(11, 6)",0,1
8,0,8,"(19, 4)",0,1
9,0,9,"(7, 20)",0,1
